## **1) Importing Python Packages for GAN**


Mengimpor paket Python untuk implementasi Generative Adversarial Networks (GAN). Dataset CIFAR-10 dan MNIST diambil dari Keras untuk pengujian model. Model Sequential dan lapisan seperti Reshape, Dense, Flatten, Conv2D, Conv2DTranspose, dan Dropout digunakan untuk membangun arsitektur model. LeakyReLU diimpor sebagai fungsi aktivasi untuk mengatasi masalah "vanishing gradients", sementara optimizer Adam dari TensorFlow dikenal karena konvergensi cepatnya. NumPy juga diimpor untuk komputasi numerik. Terakhir, direktori generated_images dibuat untuk menyimpan gambar yang dihasilkan selama pelatihan. Dengan semua komponen ini, pengguna dapat melatih model GAN untuk menghasilkan gambar dari dataset yang dipilih.

In [ ]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout

# Import LeakyReLU directly from keras.layers
from keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images

## **2) Parameters for Neural Networks & Data**

Mendefinisikan parameter penting untuk jaringan saraf dalam Generative Adversarial Networks (GAN). Dimensi gambar ditetapkan dengan lebar dan tinggi masing-masing 32 piksel dan 3 saluran untuk gambar warna RGB, disimpan dalam variabel img_shape. Dimensi laten (latent_dim) diatur sebesar 100, yang digunakan sebagai vektor input untuk generator. Optimizer Adam diinisialisasi dengan laju pembelajaran 0.0002, mengontrol pembaruan bobot selama pelatihan.

In [ ]:
img_width = 32
img_height = 32
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0002)

## **3) Building Generator**





Mendefinisikan fungsi build_generator() untuk membangun model generator dalam Generative Adversarial Networks (GAN). Model ini menggunakan arsitektur Sequential dan dimulai dengan lapisan Dense yang menerima input dengan dimensi laten (latent_dim) dan menghasilkan 256 * 4 * 4 neuron. Fungsi aktivasi LeakyReLU ditambahkan untuk memberikan non-linearitas pada model. Selanjutnya, lapisan Reshape mengubah bentuk output menjadi (4, 4, 256).

In [ ]:
def build_generator():
    model = Sequential()

    # Create first layer, to receive the input
    model.add(Dense(256 * 4 * 4, input_dim = latent_dim))
    # 256 * 8 * 8; for upscaling the layers,
    # initial shape to construct into final shape

    # Create default activation function
    model.add(LeakyReLU(alpha = 0.2))

    # Create reshape layer
    model.add(Reshape((4, 4,256)))
    # 8,8,256 ; reffers to first layer

    # Adding more layers for neurons and better result
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha= 0.2))
    # (4,4) >> filter size
    # strides = (2,2) >> Convolutional layers, that how NN understand images

    # Create Final output layer and forming image shape
    # the shape (3, (3,3)) reffers to image shape :
    #    >>>  img_shape = (img_width, img_height, channels)
    model.add(Conv2D(3, (3,3), activation= 'tanh', padding = 'same'))

    #
    model.summary()
    return model

generator = build_generator()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 4096)                │         413,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 8, 8, 128)           │         524,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 16, 16, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 32, 32, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 32, 3)           │           3,459 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,466,115 (5.59 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 0 (0.00 B)

## **4) Building Discriminator**

Membangun model discriminator untuk GAN, yang bertugas mengidentifikasi apakah gambar yang dihasilkan adalah asli atau palsu. Dimulai dengan layer Conv2D untuk memproses gambar input (img_shape) dan mengenali fitur dengan filter 64, kernel (3,3), dan padding same. Kemudian, LeakyReLU ditambahkan sebagai aktivasi untuk menambahkan non-linearitas pada setiap lapisan konvolusi.

Beberapa lapisan Conv2D dan LeakyReLU tambahan ditambahkan untuk menangkap lebih banyak detail, diakhiri dengan lapisan Flatten untuk meratakan data agar bisa masuk ke lapisan berikutnya.

In [ ]:
def build_discriminator():
    model = Sequential()

    # Create input layer and filter and stride layer. That makes NN understand image
    model.add(Conv2D(64, (3,3), padding = 'same', input_shape = img_shape))

    # Adding activation function
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Conv2D(128, (3,3), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Conv2D(128, (3,3), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Conv2D(256, (3,3), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Flatten())

    model.add(Dropout(0.4))

    # Create output layer
    model.add(Dense(1, activation = 'sigmoid'))

    model.summary()
    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_6 (LeakyReLU)            │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_7 (LeakyReLU)            │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 262144)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 262144)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │         262,145 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 780,545 (2.98 MB)

 Trainable params: 780,545 (2.98 MB)

 Non-trainable params: 0 (0.00 B)

## **5) Connecting Neural Networks to build GAN**

Menghubungkan model generator dan discriminator untuk membentuk Generative Adversarial Network (GAN) lengkap. Pertama, model Sequential untuk GAN dibuat, dan discriminator.trainable diatur ke False agar bobotnya tidak diperbarui saat melatih GAN (sehingga hanya generator yang belajar). Model GAN kemudian menambahkan generator dan discriminator sebagai lapisan berurutan. Terakhir, model ini dikompilasi menggunakan fungsi loss binary_crossentropy dan optimizer Adam, yang memungkinkan GAN belajar menghasilkan gambar yang semakin realistis selama pelatihan.

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
GAN.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 32, 32, 3)           │       1,466,115 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 1)                   │         780,545 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,246,660 (8.57 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 780,545 (2.98 MB)

## **6) Outputting Images**


Pengaturan untuk output dari program ini

In [ ]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    # print("%.8f" % save_name)

    # Rescale images 0 - 1
    # gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = (gen_imgs + 1) / 2.0
    # gen_imgs = gen_imgs * 255

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

## **7) Training GAN**

Training model GAN yang sudah kita build

In [ ]:
def train(epochs, batch_size = 16, save_interval = 200):
  (X_train, _), (_, _) = cifar10.load_data()

  # Rescaling the data
  X_train = X_train / 127.5 -1.

  bat_per_epo = int(X_train.shape[0] / batch_size)

  # Create Y label for NN
  valid = np.ones((batch_size,1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range (epochs) :
    for j in range(bat_per_epo) :
      #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      # Generate Fakes Images
      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      # Train Discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0, 1, (batch_size, latent_dim))

      # Inverse Y label
      g_loss = GAN.train_on_batch(noise, valid)

      print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss[0]))
    save_imgs(epoch)

train(300, batch_size = 16, save_interval = 200)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


******* 0 [D loss: 0.682470, acc: 62.50%] [G loss: 0.686172]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


******* 0 [D loss: 0.686820, acc: 43.75%] [G loss: 0.687899]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
******* 0 [D loss: 0.687745, acc: 38.96%] [G loss: 0.688486]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
******* 0 [D loss: 0.686812, acc: 38.50%] [G loss: 0.687490]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
******* 0 [D loss: 0.687430, acc: 37.60%] [G loss: 0.688002]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
******* 0 [D loss: 0.688203, acc: 37.03%] [G loss: 0.688751]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
******* 0 [D loss: 0.688134, acc: 38.02%] [G loss: 0.688703]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
******* 0 [D loss: 0.688907, acc: 37.94%] [G loss: 0.689478]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
******* 0 [D loss: 0.689857, acc: 37.53%] [G loss: 0.690432]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
******* 0 [D loss: 0.690569, acc: 38.17%] [G loss: 0.691251]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
******* 0 [D loss: 0.691252, acc: 38.39%] [G loss: 0.691993]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
******* 0 

# *Training tidak bisa selesai karena runtime error yang terjadi pada google colab, ketika batch dan epochs sudah disesuaikan tetap tidak bisa selesai training jadi belum bisa lanjut ke kode berikutnya.*

In [ ]:
noise = np.random.normal(0, 1, (1,latent_dim))
gen_imgs = generator.predict(noise)

In [ ]:
pgen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs[0])

### **8) Making GIF**

Membuat GIF dari gambar-gambar yang dihasilkan selama pelatihan GAN.

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)

  if filenames:  # Check if filenames is not empty
    image = imageio.imread(filenames[-1])
    writer.append_data(image)